In [ ]:
from Bio import Entrez
from Bio import SeqIO

Entrez.email = "Your.Name.Here@example.org"
handle = Entrez.efetch(db="nucleotide", id="DQ398041", rettype="gb", retmode="gb")

In [ ]:
from Bio import Entrez
from Bio import SeqIO
import pandas as pd
import csv

#Load csv into dataframe, drop extra columns and N/A accession numbers, reset index
phages_df = pd.read_csv('CombinedData.csv')
#phages_df = phages_df[['Phage Name', 'Accession #']]
phages_df = phages_df.dropna(subset=['Accession #'])
phages_df = phages_df.reset_index(drop=True)

#Entrez email and API
#Entrez.email = ''
#Entrez.api_key = ''

In [269]:
shortened_protein_dict = {}
shortened_protein_translations = {}

In [271]:
#Filter out proteins with very few phages to make matching process quicker
for key in protein_dict.keys():
    if len(protein_dict[key]) > 9:
        shortened_protein_dict[key] = protein_dict[key]
        shortened_protein_translations[key] = protein_translations[key]


print(len(shortened_protein_dict))
print(len(shortened_protein_translations))

2325
2325


In [ ]:
#Iterate through phages_df
for i in range(2602, 2627):
    phage_name = phages_df['Phage Name'][i]
    accession_number = phages_df['Accession #'][i]
    #Get proteins from genbank
    handle = Entrez.efetch(db="nucleotide", id=accession_number, rettype="gb", retmode="text")
    #matches = 0
    for record in SeqIO.parse(handle, 'genbank'):
        #Iterate through proteins
        for seq_feature in record.features:
            match_found = False
            if seq_feature.type == 'CDS':
                for key in shortened_protein_translations.keys():
                    #If protein alreday exists in dict, append phage name to protein_dict
                    translation = seq_feature.qualifiers['translation'][0]
                    if translation == shortened_protein_translations[key]:
                        if phage_name not in protein_dict[key]:
                            protein_dict[key].append(phage_name)
                            match_found = True
                            #matches += 1
                            break
                #If protein does not already exist in dict, add protein to both protein_dict and protein_translations
                if match_found == False:
                    #Arbitrary protein numbering system to make referencing proteins easier
                    new_key = f"Protein {len(protein_translations) + 1}"
                    protein_translations[new_key] = seq_feature.qualifiers['translation'][0]
                    protein_dict[new_key] = [phage_name]
    print(i)
    #print(f"{matches} matches found")

In [285]:
print(len(phages_df))

2627


In [ ]:
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 

#Experimentation with fuzzy string comparison
string1 = 'msdyrgthta ptddgysvpl dslsdhtadv yerpqdhhgg ddtetmvavl sargnpharv tiyravphgv tainagdwvs lsegyardhg dtegmtvlra tvrasqvwwd gnslyeygyd gpdtegqta'
string2 = 'msdyrgthta ptndgygvsl dslsdhtadv yarprdhhgg ddtetmtava racgnpkarv tiyravphgv ttinagdwvs lsedyarehg dtegmtvlrv tvrasqvwwd gnslyeygyd gpdtegqta'

fuzz.ratio(string1, string2)

In [296]:
#Save protein_dict to JSON file
import json
protein_dict_json = json.dumps(protein_dict)
f = open('protein_dict.json', 'w')
f.write(protein_dict_json)
f.close()

In [297]:
#Save protein_translations to JSON file
protein_translations_json = json.dumps(protein_translations)
f = open('protein_translations.json', 'w')
f.write(protein_translations_json)
f.close()

In [298]:
#Save shortened_protein_translations to JSON file
shortened_protein_dict_json = json.dumps(shortened_protein_dict)
f = open('shortened_protein_dict.json', 'w')
f.write(shortened_protein_dict_json)
f.close()

In [299]:
#Save shortened_protein_translations to JSON file
shortened_protein_translations_json = json.dumps(shortened_protein_translations)
f = open('shortened_protein_translations.json', 'w')
f.write(shortened_protein_translations_json)
f.close()

In [1]:
string1 = shortened_protein_translations['Protein 8']

for key in protein_translations:
    if fuzz.ratio(string1, protein_translations[key]) > 95:
        print(protein_dict[key])

NameError: name 'shortened_protein_translations' is not defined

In [ ]:
#Fuzzy string comparison, too slow

#Iterate through phages_df
#for i in range(57, 2000):
    print(i)
    phage_name = phages_df['Phage Name'][i]
    accession_number = phages_df['Accession #'][i]
    #Get proteins from genbank
    handle = Entrez.efetch(db="nucleotide", id=accession_number, rettype="gb", retmode="gb")
    for record in SeqIO.parse(handle, 'genbank'):
        #Iterate through proteins
        for seq_feature in record.features:
            match_found = False
            if seq_feature.type == 'CDS':
                for key in protein_translations.keys():
                    #If protein alreday exists in dict (using fuzzy string comparison), append phage name to protein_dict
                    if fuzz.ratio(seq_feature.qualifiers['translation'][0], protein_translations[key]) > 85:
                        if phage_name not in protein_dict[key]:
                            protein_dict[key].append(phage_name)
                            match_found = True
                #If protein does not already exist in dict, add protein to both protein_dict and protein_translations
                if match_found == False:
                    #Arbitrary protein numbering system to make referencing proteins easier
                    new_key = f"Protein {len(protein_translations) + 1}"
                    protein_translations[new_key] = seq_feature.qualifiers['translation'][0]
                    protein_dict[new_key] = [phage_name]